In [1]:
!pip install pyyaml h5py

In [2]:
#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive

# This will prompt for authorization.
#drive.mount('content/')

In [3]:
# Testing the AI
# Installing Keras
# conda install -c conda-forge keras

# Importing the libraries and the other python files
import os
import numpy as np
import random as rn
from keras.models import load_model 
import step1_env as environment
import pandas as pd

In [5]:
# Setting seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '0' 
np.random.seed(42)
rn.seed(12345)
 
# SETTING THE PARAMETERS
run = 'model' ## Name of the run that you are doing. Used to load the model file and save the output
number_actions = 5
direction_boundary = (number_actions - 1) / 2
temperature_step = 1.5

# BUILDING THE ENVIRONMENT BY SIMPLY CREATING AN OBJECT OF THE ENVIRONMENT CLASS
env = environment.Environment(optimal_temperature = (18.0, 24.0), initial_month = 0,
								initial_number_users = 20, initial_rate_data = 30)

# LOADING A PRE-TRAINED BRAIN
model = load_model(run + 'h5') 

# CHOOSING THE MODE
train = False

In [6]:
results = []

In [7]:
# RUNNING A 1 YEAR SIMULATION IN INFERENCE MODE
env.train = train
current_state, _, _ = env.observe()
for timestep in range(0, 12 * 30 * 24 * 60):
  q_values = model.predict(current_state)
  action = np.argmax(q_values[0])
  
  if (action - direction_boundary < 0): 
    direction = -1
  else:
    direction = 1

  energy_ai = abs(action - direction_boundary) * temperature_step 
  next_state, reward, game_over = env.update_env(direction, \
                                                 energy_ai, \
                                                 int(timestep / (30*24*60)))

  current_state = next_state
  print("\n")
  print(timestep)
  print("Total Energy spent with an AI: {:.0f}".format(env.total_energy_ai)) 
  print("Total Energy spent with no AI: {:.0f}".format(env.total_energy_noai))
  print("ENERGY SAVED: {:.0f} %".format(100*(env.total_energy_noai - env.total_energy_ai)
                                     / env.total_energy_noai))
  results.append([timestep,
  env.total_energy_ai,
  env.total_energy_noai,
  (env.total_energy_noai - env.total_energy_ai)/env.total_energy_noai,
  env.intrinsic_temperature,
  env.temperature_ai,
  env.temperature_noai,
  direction,
  energy_ai])

n AI: 1575028
Total Energy spent with no AI: 2464882
ENERGY SAVED: 36 %


518212
Total Energy spent with an AI: 1575029
Total Energy spent with no AI: 2464883
ENERGY SAVED: 36 %


518213
Total Energy spent with an AI: 1575030
Total Energy spent with no AI: 2464883
ENERGY SAVED: 36 %


518214
Total Energy spent with an AI: 1575032
Total Energy spent with no AI: 2464885
ENERGY SAVED: 36 %


518215
Total Energy spent with an AI: 1575034
Total Energy spent with no AI: 2464892
ENERGY SAVED: 36 %


518216
Total Energy spent with an AI: 1575035
Total Energy spent with no AI: 2464896
ENERGY SAVED: 36 %


518217
Total Energy spent with an AI: 1575036
Total Energy spent with no AI: 2464897
ENERGY SAVED: 36 %


518218
Total Energy spent with an AI: 1575038
Total Energy spent with no AI: 2464897
ENERGY SAVED: 36 %


518219
Total Energy spent with an AI: 1575040
Total Energy spent with no AI: 2464898
ENERGY SAVED: 36 %


518220
Total Energy spent with an AI: 1575041
Total Energy spent with no AI: 2

In [8]:
# PRINTING THE TRAINING RESULTS FOR EACH EPOCH
print("\n")
print("Total Energy spent with an AI: {:.0f}".format(env.total_energy_ai)) 
print("Total Energy spent with no AI: {:.0f}".format(env.total_energy_noai))
print("ENERGY SAVED: {:.0f} %".format((env.total_energy_noai - env.total_energy_ai)
                                     / env.total_energy_noai * 100))



Total Energy spent with an AI: 1575580
Total Energy spent with no AI: 2465713
ENERGY SAVED: 36 %


In [9]:
results = np.array(results)

In [10]:
results = np.transpose(results)

In [11]:
df = pd.DataFrame()

In [12]:
results[0]

array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 5.18397e+05,
       5.18398e+05, 5.18399e+05])

In [13]:
df['timestep'] = results [0]
df['env.total_energy_ai'] = results [1]
df['total_energy_noai']  = results [2]
df['% Savings '] = results [3]
df['inttrinsic_temperature'] = results [4]
df['temp_ai'] = results [5]
df['temp_noai'] = results[6]
df['direction'] = results [7]
df['energy_ai'] = results [8]

In [19]:
# Calculate the energy usage in each time step:
energy_ai_inst = [0]
energy_noai_inst = [0]
for index in range(1, len(results[0])):
    energy_ai_inst.append(results[1][index] - results[1][index-1])
    energy_noai_inst.append(results[2][index] - results [2][index -1])
df['energy_ai_inst'] = energy_ai_inst
df['energy_noai_inst'] = energy_noai_inst
    

In [20]:
df.to_csv(run + ".csv")